# Data preparation

1) obtaining cloud-free composite for the AOI using GEE python API
2) masking composite raster by label polygons with classes:

1 - ``trees`` \
2 - ``flooded vegetation`` \
3 - ``open water`` \
4 - ``settlements`` \
5 - ``bare soil`` - a target class which dynamics is of the most interest along with ``trees`` \
6 - ``agriculture`` \
7 - ``shrubs`` \
8 - ``grass``

In [1]:
#libraries
import ee
import pandas as pd
#import altair as alt 
import numpy as np
import geemap.foliumap as geemap
import folium
import geopandas as gpd
import rasterio as rio
from rasterio.mask import mask

import requests
import json
from geojson import Point, Feature, FeatureCollection, dump

import matplotlib.pyplot as plt

In [2]:
import warnings
warnings.filterwarnings('ignore')

## Sentinel-2 summer composite download using GEE

Based on [Sentinel-2 Cloud Masking with s2cloudless](https://developers.google.com/earth-engine/tutorials/community/sentinel-2-s2cloudless) tutorial

In [3]:
#authorisation
ee.Authenticate()

Enter verification code: 4/1AbUR2VPuExgWeh7upAqRn5IDNILUUgx2-XQ8B3BYKYkyd89AM-R9zFDFcWw

Successfully saved authorization token.


In [4]:
ee.Initialize()

In [5]:
#all helpers from GEE
def get_s2_sr_cld_col(aoi, start_date, end_date):
    # Import and filter S2 SR.
    s2_sr_col = (ee.ImageCollection('COPERNICUS/S2_SR')
        .filterBounds(aoi)
        .filterDate(start_date, end_date)
        .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', CLOUD_FILTER)))

    # Import and filter s2cloudless.
    s2_cloudless_col = (ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')
        .filterBounds(aoi)
        .filterDate(start_date, end_date))

    # Join the filtered s2cloudless collection to the SR collection by the 'system:index' property.
    return ee.ImageCollection(ee.Join.saveFirst('s2cloudless').apply(**{
        'primary': s2_sr_col,
        'secondary': s2_cloudless_col,
        'condition': ee.Filter.equals(**{
            'leftField': 'system:index',
            'rightField': 'system:index'
        })
    }))

def add_cloud_bands(img):
    # Get s2cloudless image, subset the probability band.
    cld_prb = ee.Image(img.get('s2cloudless')).select('probability')

    # Condition s2cloudless by the probability threshold value.
    is_cloud = cld_prb.gt(CLD_PRB_THRESH).rename('clouds')

    # Add the cloud probability layer and cloud mask as image bands.
    return img.addBands(ee.Image([cld_prb, is_cloud]))

def add_shadow_bands(img):
    # Identify water pixels from the SCL band.
    not_water = img.select('SCL').neq(6)

    # Identify dark NIR pixels that are not water (potential cloud shadow pixels).
    SR_BAND_SCALE = 1e4
    dark_pixels = img.select('B8').lt(NIR_DRK_THRESH*SR_BAND_SCALE).multiply(not_water).rename('dark_pixels')

    # Determine the direction to project cloud shadow from clouds (assumes UTM projection).
    shadow_azimuth = ee.Number(90).subtract(ee.Number(img.get('MEAN_SOLAR_AZIMUTH_ANGLE')));

    # Project shadows from clouds for the distance specified by the CLD_PRJ_DIST input.
    cld_proj = (img.select('clouds').directionalDistanceTransform(shadow_azimuth, CLD_PRJ_DIST*10)
        .reproject(**{'crs': img.select(0).projection(), 'scale': 100})
        .select('distance')
        .mask()
        .rename('cloud_transform'))

    # Identify the intersection of dark pixels with cloud shadow projection.
    shadows = cld_proj.multiply(dark_pixels).rename('shadows')

    # Add dark pixels, cloud projection, and identified shadows as image bands.
    return img.addBands(ee.Image([dark_pixels, cld_proj, shadows]))

def add_cld_shdw_mask(img):
    # Add cloud component bands.
    img_cloud = add_cloud_bands(img)

    # Add cloud shadow component bands.
    img_cloud_shadow = add_shadow_bands(img_cloud)

    # Combine cloud and shadow mask, set cloud and shadow as value 1, else 0.
    is_cld_shdw = img_cloud_shadow.select('clouds').add(img_cloud_shadow.select('shadows')).gt(0)

    # Remove small cloud-shadow patches and dilate remaining pixels by BUFFER input.
    # 20 m scale is for speed, and assumes clouds don't require 10 m precision.
    is_cld_shdw = (is_cld_shdw.focalMin(2).focalMax(BUFFER*2/20)
        .reproject(**{'crs': img.select([0]).projection(), 'scale': 20})
        .rename('cloudmask'))

    # Add the final cloud-shadow mask to the image.
    return img_cloud_shadow.addBands(is_cld_shdw)

# Define a method for displaying Earth Engine image tiles to a folium map.
def add_ee_layer(self, ee_image_object, vis_params, name, show=True, opacity=1, min_zoom=0):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name=name,
        show=show,
        opacity=opacity,
        min_zoom=min_zoom,
        overlay=True,
        control=True
        ).add_to(self)

# Add the Earth Engine layer method to folium.
folium.Map.add_ee_layer = add_ee_layer

def apply_cld_shdw_mask(img):
    # Subset the cloudmask band and invert it so clouds/shadow are 0, else 1.
    not_cld_shdw = img.select('cloudmask').Not()

    # Subset reflectance bands and update their masks, return the result.
    return img.select('B.*').updateMask(not_cld_shdw)

In [6]:
#bbox of the area of interest

f_kola = open('data/monche_bbox.geojson')
kola_bbox = json.load(f_kola)
aoi_poly = ee.Geometry.MultiPolygon(kola_bbox['features'][0]['geometry']['coordinates'])

Map = geemap.Map()
Map.addLayer(aoi_poly,{},'AOI_Monchegorsk')
Map.setCenter(32.8, 67.92,  11)
Map

In [ ]:
#settings of example
AOI = aoi_poly
START_DATE = '2020-06-15'
END_DATE = '2020-08-25'
CLOUD_FILTER = 60
CLD_PRB_THRESH = 50
NIR_DRK_THRESH = 0.15
CLD_PRJ_DIST = 1
BUFFER = 50

In [ ]:
#getting cloud-free median composite for AOI and period of time
s2_sr_cld_col_eval = get_s2_sr_cld_col(AOI, START_DATE, END_DATE)
s2_sr_cld_col = get_s2_sr_cld_col(AOI, START_DATE, END_DATE)
s2_sr_median = (s2_sr_cld_col.map(add_cld_shdw_mask)
                             .map(apply_cld_shdw_mask)
                             .median())
kola_2020 = s2_sr_median.clip(AOI)

# Create a folium map object.
center = AOI.centroid(10).coordinates().reverse().getInfo()
m = folium.Map(location=center, zoom_start=12)

# Add layers to the folium map.
m.add_ee_layer(kola_2020,
                {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 2500, 'gamma': 1.1},
                'S2 cloud-free mosaic', True, 1, 9)

# Add a layer control panel to the map.
m.add_child(folium.LayerControl())

# Display the map.
display(m)

In [ ]:
#median composite writing to tiff file -- warning! it will be writed to the gdrive

#task = ee.batch.Export.image.toDrive(image=kola_2020,
#                                     description='2020_kola_median_composite',
#                                     scale=30,
#                                     #region=AOI,
#                                     fileNamePrefix='2020_kola_median_composite',
#                                     crs='EPSG:4326',
#                                     fileFormat='GeoTIFF')
#task.start()
#task.status()

## Getting dataset for predictions

In [ ]:
#helper indices function

def get_spectral_indices(df):
    blue = df['B2']
    green = df['B3']
    red = df['B4']
    nir = df['B8']
    thermal2 = df['B11']
    
    df['ndvi'] = (nir-red)/(nir+red)
    df['evi'] = 2.5 * ((nir - red) / (nir + 6 * red - 7.5 * blue + 1))
    df['savi'] = (nir - red) / (nir + red + 0.428) * (1.428)
    df['msi'] = thermal2/nir
    df['bsi'] = ((thermal2+red)-(nir+blue))/((thermal2+red)+(nir+blue))
    
    return df

In [ ]:
label_g = open('data/label_data.geojson')
labels = json.load(label_g)
tiff_path = 'data/2020_kola_median_composite.tif'

In [ ]:
#getting dataset of band values in pixels of polygons 
general = pd.DataFrame()


for row in labels['features']:
    with rio.open(tiff_path, 'r+') as dataset:
        df = pd.DataFrame(rio.mask.mask(dataset, 
                                    [row['geometry']], 
                                    all_touched=True,
                                    crop=True)[0].reshape([12,-1]).T, 
                      columns=list(dataset.descriptions))
        df['id'] = row['properties']['id']
        general = pd.concat([general, df])
        general = general[(general[(list(dataset.descriptions))] > 0).any(1)]
        general = general.reset_index().iloc[:,1:]
        get_spectral_indices(general)
    
general  

🙌